In [24]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import random
import json
import os
import shutil
from pathlib import Path
import os.path
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
def coco_to_yolo_bb(x1, y1, w, h, image_w, image_h):
    #"bbox": [x,y,width,height], (COCO)
    #x_center y_center width height (YOLO)
    return [((2*x1 + w)/(2*image_w)) , ((2*y1 + h)/(2*image_h)), w/image_w, h/image_h]

In [25]:
def getCatData():
    # display COCO categories and supercategories
    cats = coco.loadCats(coco.getCatIds())
    cat_map = {}
    for i,cat in enumerate(cats):
        cat_map[cat['id']] = i
    nms=[cat['name'] for cat in cats]
    catIds = coco.getCatIds()
    return catIds, cat_map

In [26]:
def get_imgs_from_each_category(catIds, numImgs=None):
    imgIds = []
    for catId in catIds:
        imgIds_current_batch = coco.getImgIds(catIds=[catId])[:numImgs]
        if(len(imgIds_current_batch) < 30):
            print('ERROR! Not 30 images in this category!')
        imgIds = imgIds + imgIds_current_batch
    return imgIds

In [33]:
"""
getImgIds() -> list[imgId] 
loadImgs(ids : [imgId]) -> list[img]
getAnnIds(imgIds: list[imgId]) -> list[annId]
loadAnns(ids: list[annId]) -> list[ann]

*One imgId can be multiple annIds
*One annId is one ann


#imgIds need to have annotations and those annotations bboxes must not be negative
#write_label_files need
    #filename, which are in imgs
    #catIds, which are in anns
    #bboxes, which are in anns
"""
def get_random_img_ids(numImgs):
    imgIds = coco.getImgIds()
    random.shuffle(imgIds)
    imgIds = list(filter(imgId_has_annotation, imgIds))
    #imgIds = [imgId for imgId in imgIds if imgId_has_annotation(imgId)]
    #imgIds = [imgId for imgId in imgIds if imgId_has_positive_bboxes(imgId)]
    #imgIds = filter(imgId_has_annotation, imgIds)
    #imgIds = filter(imgId_has_positive_bboxes, imgIds)
    imgIds = list(filter(imgId_has_positive_bboxes, imgIds))
    return imgIds[:numImgs]

def get_img_ids_per_cat(numImgs_per_cat):
    catIds = coco.getCatIds()
    imgIds_total = []
    for catId in catIds:
        imgIds_cat = coco.getImgIds(catIds=[catId])
        #imgIds = coco.getImgIds()
        random.shuffle(imgIds_cat)
        imgIds_current = []
        for imgId in imgIds_cat:
            if imgId_has_annotation(imgId) and imgId_has_positive_bboxes(imgId):
                imgIds_current.append(imgId)
            if(len(imgIds_current) == numImgs_per_cat):
                break
        imgIds_total = imgIds_total + imgIds_current
    return imgIds_total

def imgId_has_positive_bboxes(imgId):
    annIds = coco.getAnnIds(imgIds=imgId)
    anns = coco.loadAnns(annIds)
    for ann in anns:
        negative_bbox = False 
        for bbox_val in ann['bbox']:
            if bbox_val < 0:
                negative_bbox = True
        if negative_bbox == True:
            return False
    return True

def imgId_has_annotation(imgId):
    annIds = coco.getAnnIds(imgIds=imgId)
    if(len(annIds) > 0):
        return True
    else:
        return False

#def write_label_files(numImgs_per_cat):
def write_label_files(numImgs, cat_map, catIds, label_dir1):
    """
    Some images:
    *Do not have annotations.
    *Have negative bboxes
    """
    #imgIds = get_img_ids_per_cat(numImgs_per_cat)
    imgIds = get_random_img_ids(numImgs)
    print(len(imgIds) == 98)

    for imgId in imgIds:
        img = coco.loadImgs(imgId)[0] #If only 1 imgId, len() = 1
        label_dir = Path(label_dir1)
                        #'/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_val1/dataset/labels'
        filename = img['file_name'].split('.')[0] + '.txt'
        with open(label_dir/filename, 'w') as file:
            annIds = coco.getAnnIds(imgIds = imgId, catIds=catIds)
            #assert len(annIds) != 0
            anns = coco.loadAnns(annIds)
            #if not anns:
                #print('no anns for this file')
            #print(len(anns))
            #print('-----------------------------')
            #anns = remove_negative_bbox_anns(anns)
            for i,ann in enumerate(anns):
                x_center, y_center, width, height = coco_to_yolo_bb(*ann['bbox'], img['width'], img['height'])
                #print(label_dir/filename)
                #print(x_center, y_center, width, height)
                #print('\n')
                file.write(f'{cat_map[ann["category_id"]]} {x_center} {y_center} {width} {height}\n')
    return

def copy_images_to_dataset_dir(src_image_dir, dest_image_dir, image_extension, src_label_dir):
    """
    x number of labels are written to the labels dir by write_label_files(x)
    This function copies the corresponding images into the sibling images dir, completing the dataset folder for YOLO training
    """
    fs = os.listdir(src_label_dir)
    for f in fs:
        src = str(src_image_dir/f.split('.')[0]) + image_extension
        dest = str(dest_image_dir/ f.split('.')[0]) + image_extension
        if os.path.isfile(src):
            shutil.copyfile(src, dest)
        else:
            print(f'{src} is not a file')
    return

#Currently outputting 473 instead of 480 (4*120)
#So one of more categories has fewer than 120 valid images, it *seems*
if __name__ == "__main__":
    #os.system('rm -rf /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/*; rm -rf /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels/*')


    #region Training -----------------------------------------------------
    #each method I've written uses coco
    #initialize COCO api for instance annotations
    #coco=COCO('/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_train1/nightowls_validation')#
    #coco=COCO('/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_training.json')
    #catIds, cat_map = getCatData()
    #write_label_files(480, cat_map, catIds, '/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_train1/dataset/labels') 
    #copy_images_to_dataset_dir(Path('/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation'),Path('/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_val1/dataset/images/'), '.png', '/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_train1/dataset/labels') #dataset/images
    #endregion


    #Validation -----------------------------------------------------
    coco=COCO('/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation.json')
    catIds, cat_map = getCatData()
    write_label_files(98, cat_map, catIds, '/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_val1/dataset/labels') 
    copy_images_to_dataset_dir(Path('/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation'),Path('/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_val1/dataset/images/'), '.png', '/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_val1/dataset/labels') #dataset/images

    #region 
    #coco=COCO('/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_train1/nightowls_validation')
    #write_label_files(187) #dataset/labels
    #copy_images_to_dataset_dir(Path('/Users/azakaria/Documents/yolo datasets/nightowls/nightowls_validation'),Path('/Users/azakaria/Code/openmpf-yolo-training/nightowls/nightowls_val1/dataset/images/'), '.png') #dataset/images
    #os.system('/usr/local/bin/docker exec -ti 5dcf6e04dbba rm -rf /usr/src/datasets/nightowls_val/images')
    #os.system('/usr/local/bin/docker exec -ti 5dcf6e04dbba rm -rf /usr/src/datasets/nightowls_val/labels')
    #os.system('/usr/local/bin/docker cp /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/. 5dcf6e04dbba:/usr/src/datasets/nightowls_val')
    #print('ls /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images | wc -l ; ls /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels | wc -l')
    #os.system('ls /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images | wc -l ; ls /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels | wc -l')
    #endregion

#region
#rm -rf /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/*; rm -rf /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels/*
#docker cp /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/. 5dcf6e04dbba:/usr/src/datasets/nightowls_val
#ls /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images | wc -l ; ls /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels | wc -l
#endregion


#region useful test code for single imgId
#imgId = 7000058
""""
imgIds = coco.getImgIds()
imgId = imgIds[2]
img = coco.loadImgs(ids = imgId)[0]
annIds = coco.getAnnIds(imgIds = imgId)
anns = coco.loadAnns(annIds)
print(coco)

fp = f'/Users/azakaria/Downloads/nightowls_validation/{img["file_name"]}' 
I = io.imread(fp)
plt.axis('off')
plt.imshow(I)
plt.show()

plt.imshow(I); plt.axis('off')
coco.showAnns(anns, draw_bbox=True)
"""
#endregion

loading annotations into memory...
Done (t=0.30s)
creating index...
index created!
True


'"\nimgIds = coco.getImgIds()\nimgId = imgIds[2]\nimg = coco.loadImgs(ids = imgId)[0]\nannIds = coco.getAnnIds(imgIds = imgId)\nanns = coco.loadAnns(annIds)\nprint(coco)\n\nfp = f\'/Users/azakaria/Downloads/nightowls_validation/{img["file_name"]}\' \nI = io.imread(fp)\nplt.axis(\'off\')\nplt.imshow(I)\nplt.show()\n\nplt.imshow(I); plt.axis(\'off\')\ncoco.showAnns(anns, draw_bbox=True)\n'

In [28]:
print(catIds, cat_map)
imgIds = get_random_img_ids(98)
t = []
for imgId in imgIds:
    #annIds = coco.getAnnIds(imgIds = imgId, catIds=catIds)
    annIds = coco.getAnnIds(imgIds = imgId)
    if not annIds:
        print(f'No annotation: {imgId}')
        t.append(annIds)
        #print('not annIds')
    else:
        print(f'Yes annotation: {imgId}')
print(len(t))

[1, 2, 3, 4] {1: 0, 2: 1, 3: 2, 4: 3}
Yes annotation: 7026977
Yes annotation: 7030024
Yes annotation: 7028256
Yes annotation: 7021288
Yes annotation: 7022342
Yes annotation: 7044662
Yes annotation: 7028452
Yes annotation: 7027661
Yes annotation: 7021328
Yes annotation: 7000204
Yes annotation: 7057491
Yes annotation: 7057143
Yes annotation: 7034849
Yes annotation: 7043831
Yes annotation: 7012173
Yes annotation: 7000537
Yes annotation: 7024130
Yes annotation: 7056804
Yes annotation: 7029470
Yes annotation: 7020873
Yes annotation: 7021969
Yes annotation: 7000278
Yes annotation: 7031933
Yes annotation: 7021367
Yes annotation: 7045126
Yes annotation: 7028065
Yes annotation: 7023552
Yes annotation: 7046533
Yes annotation: 7027072
Yes annotation: 7004835
Yes annotation: 7000177
Yes annotation: 7028281
Yes annotation: 7018485
Yes annotation: 7024398
Yes annotation: 7021754
Yes annotation: 7001137
Yes annotation: 7028060
Yes annotation: 7034821
Yes annotation: 7017111
Yes annotation: 7030003
Ye